In [1]:
!git clone https://github.com/davide-coccomini/Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection.git
%cd Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection


Cloning into 'Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 162 (delta 28), reused 14 (delta 14), pack-reused 123 (from 1)
Receiving objects: 100% (162/162), 1.10 MiB | 2.64 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/workspace/Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection


In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [3]:

binshilin_genreal_old_deepfakes_path = kagglehub.dataset_download('binshilin/final-dataset-genreal')

print('Data source import complete.')


100%|██████████| 8.26G/8.26G [01:58<00:00, 75.0MB/s]

Extracting files...


Data source import complete.


In [4]:
!pip install Pillow==9.5.0 --force-reinstall
!pip install efficientnet_pytorch tqdm
!pip install opencv-python
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 92.6 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16478 sha256=e024bc6f2f3d2080335b7d58bce30719df1bd08ce70261a5124fbe49031a7bdf
  Stored in directory: /root/.cache/pip/wheels/8b/6f/9b/231a832f811ab6ebb1b32455b177ffc6b8b1cd8de19de70c09
Successfully built efficientnet_pytorch

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 67.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To

In [ ]:
import os
import shutil

# Root dataset directory
dataset_root = '/path/to/data_split'

# Threshold for minimum number of frames
MIN_FRAMES = 16

# Function to remove subfolders with fewer than MIN_FRAMES images
def clean_short_sequences(root_path):
    removed_count = 0
    for root, dirs, files in os.walk(root_path):
        # Only go one level deep from each person folder
        if 'person_' in os.path.basename(root):
            image_files = [f for f in os.listdir(root) if f.endswith(('.jpg', '.png'))]
            if len(image_files) < MIN_FRAMES:
                print(f"Removing {root} ({len(image_files)} frames)")
                shutil.rmtree(root)
                removed_count += 1
    print(f"✅ Removed {removed_count} folders with < {MIN_FRAMES} frames under {root_path}\n")

# Apply it to all four paths
for split in ['train', 'test']:
    for label in ['real', 'fake']:
        path = os.path.join(dataset_root, split, label)
        clean_short_sequences(path)


In [ ]:
# TRIAL 1
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import numpy as np
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
import warnings
warnings.filterwarnings("ignore")

# ----------- Dataset Class with Sequential Batching -----------
class FaceSequenceDataset(Dataset):
    def __init__(self, root_dir, transform=None, max_frames=32, min_frames=16, frame_interval=1):
        self.samples = []
        self.transform = transform
        self.label_map = {'original': 0, 'fake': 1}
        self.max_frames = max_frames
        self.min_frames = min_frames
        self.frame_interval = frame_interval

        for label in ['original', 'fake']:
            class_dir = os.path.join(root_dir, label)
            if not os.path.exists(class_dir):
                continue
            for folder in os.listdir(class_dir):
                folder_path = os.path.join(class_dir, folder)
                if not os.path.isdir(folder_path):
                    continue
                for person_folder in os.listdir(folder_path):
                    person_path = os.path.join(folder_path, person_folder)
                    if not os.path.isdir(person_path):
                        continue
                    frame_paths = sorted(glob.glob(os.path.join(person_path, '*.jpg')))
                    
                    # Create sequential batches from the frame sequence
                    if len(frame_paths) >= self.min_frames:
                        self._create_sequential_batches(frame_paths, self.label_map[label])

    def _create_sequential_batches(self, frame_paths, label):
        """Create sequential batches of frames from a single video sequence"""
        total_frames = len(frame_paths)
        start_idx = 0
        
        while start_idx < total_frames:
            end_idx = min(start_idx + self.max_frames, total_frames)
            batch_frames = frame_paths[start_idx:end_idx]
            
            # Check if this batch meets our criteria
            if len(batch_frames) >= self.min_frames:
                self.samples.append((batch_frames, label))
            # If less than min_frames, discard this batch
            
            start_idx += self.max_frames

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        frame_paths, label = self.samples[idx]
        
        images = []
        for frame_path in frame_paths:
            try:
                img = Image.open(frame_path).convert('RGB')
                if self.transform:
                    img = self.transform(img)
                images.append(img)
            except Exception as e:
                print(f"Error loading {frame_path}: {e}")
                # Use a black image as fallback
                if self.transform:
                    img = self.transform(Image.new('RGB', (224, 224), (0, 0, 0)))
                else:
                    img = torch.zeros(3, 224, 224)
                images.append(img)
        
        # Pad sequence if needed (for sequences with min_frames <= length < max_frames)
        while len(images) < self.max_frames:
            images.append(torch.zeros_like(images[0]))
            
        images = torch.stack(images)  # (T, C, H, W)
        return images, torch.tensor(label, dtype=torch.long)

# ----------- Improved Model with Proper Spatial-Temporal Processing -----------
class SpatialTemporalDeepfakeDetector(nn.Module):
    def __init__(self, num_classes=2, max_frames=32, feature_dim=1408):
        super().__init__()
        
        # Spatial feature extractor
        self.spatial_encoder = EfficientNet.from_pretrained('efficientnet-b2')
        self.spatial_encoder._fc = nn.Identity()
        
        # Spatial feature projection
        self.spatial_projection = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # Temporal modeling with LSTM + Transformer
        self.temporal_lstm = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=2,
            batch_first=True,
            dropout=0.3,
            bidirectional=True
        )
        
        # Temporal attention
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=512,  # bidirectional LSTM output
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        # Positional encoding for temporal sequences
        self.pos_encoding = nn.Parameter(torch.randn(max_frames, 512))
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):  # x: (B, T, C, H, W)
        B, T, C, H, W = x.shape
        
        # Extract spatial features for each frame
        x_reshaped = x.view(B * T, C, H, W)
        spatial_features = self.spatial_encoder.extract_features(x_reshaped)  # (B*T, 1408, h, w)
        
        # Project spatial features
        spatial_features = self.spatial_projection(spatial_features)  # (B*T, 512)
        spatial_features = spatial_features.view(B, T, -1)  # (B, T, 512)
        
        # Add positional encoding
        spatial_features = spatial_features + self.pos_encoding[:T].unsqueeze(0)
        
        # Temporal modeling with LSTM
        lstm_out, _ = self.temporal_lstm(spatial_features)  # (B, T, 512)
        
        # Temporal attention
        attended_features, attention_weights = self.temporal_attention(
            lstm_out, lstm_out, lstm_out
        )  # (B, T, 512)
        
        # Global temporal pooling
        temporal_representation = torch.mean(attended_features, dim=1)  # (B, 512)
        
        # Classification
        output = self.classifier(temporal_representation)
        
        return output

# ----------- Training Utilities -----------
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1

def analyze_dataset_composition(dataset):
    """Analyze how many samples were created from sequences of different lengths"""
    print(f"Total training samples created: {len(dataset)}")
    
    # Count class distribution
    original_count = sum(1 for _, label in dataset.samples if label == 0)
    fake_count = sum(1 for _, label in dataset.samples if label == 1)
    
    print(f"Original samples: {original_count}")
    print(f"Fake samples: {fake_count}")
    print(f"Class ratio (fake/original): {fake_count/original_count:.2f}")

def train(model, train_loader, val_loader, device, num_epochs=30, checkpoint_path='best_model.pth'):
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 1.5]).to(device))  # Weight for class imbalance
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    scaler = GradScaler()

    best_val_f1 = 0.0
    early_stop_counter = 0
    patience = 7

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, all_preds, all_labels = 0, [], []

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, (inputs, labels) in enumerate(progress_bar):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # Update progress bar
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            # Clear cache periodically
            if batch_idx % 50 == 0:
                torch.cuda.empty_cache()

        train_acc, train_prec, train_rec, train_f1 = calculate_metrics(all_labels, all_preds)
        avg_train_loss = train_loss / len(train_loader)
        
        print(f"Train - Loss: {avg_train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}")

        # Validation phase
        model.eval()
        val_loss, all_preds, all_labels = 0, [], []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validating"):
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                
                with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_acc, val_prec, val_rec, val_f1 = calculate_metrics(all_labels, all_preds)
        avg_val_loss = val_loss / len(val_loader)
        
        print(f"Val - Loss: {avg_val_loss:.4f}, Acc: {val_acc:.4f}, Prec: {val_prec:.4f}, Rec: {val_rec:.4f}, F1: {val_f1:.4f}")
        print("-" * 60)

        scheduler.step(avg_val_loss)

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_f1': val_f1,
            'val_loss': avg_val_loss
        }
        torch.save(checkpoint, f"checkpoint_epoch_{epoch+1}.pth")
        
        # Save best model based on F1 score
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(checkpoint, checkpoint_path)
            print(f"New best model saved with F1: {val_f1:.4f}")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping triggered after {patience} epochs without improvement.")
                break
        
        # Clear cache
        torch.cuda.empty_cache()

    print(f"Training completed. Best validation F1: {best_val_f1:.4f}")

# ----------- Main Execution -----------
if __name__ == '__main__':
    # Paths
    train_dir = '/root/.cache/kagglehub/datasets/binshilin/final-dataset-genreal/versions/1/train'
    test_dir = '/root/.cache/kagglehub/datasets/binshilin/final-dataset-genreal/versions/1/test'

    # Enhanced data augmentation for training
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1)
        ], p=0.7),
        transforms.RandomGrayscale(p=0.05),
        transforms.RandomRotation(10),
        transforms.RandomApply([
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.5))
        ], p=0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
    ])

    # Simple transform for validation
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create datasets with sequential batching
    print("Creating training dataset with sequential batching...")
    train_dataset = FaceSequenceDataset(train_dir, transform=train_transform, max_frames=32, min_frames=16)
    
    print("Creating validation dataset with sequential batching...")
    val_dataset = FaceSequenceDataset(test_dir, transform=val_transform, max_frames=32, min_frames=16)

    # Analyze dataset composition
    print("\n=== Training Dataset Analysis ===")
    analyze_dataset_composition(train_dataset)
    print("\n=== Validation Dataset Analysis ===")
    analyze_dataset_composition(val_dataset)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=2,  # Adjust based on your GPU memory
        shuffle=True, 
        num_workers=4, 
        pin_memory=True,
        drop_last=True
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=2, 
        shuffle=False, 
        num_workers=4, 
        pin_memory=True
    )

    # Initialize model and training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nUsing device: {device}")
    
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    model = SpatialTemporalDeepfakeDetector().to(device)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

    print(f"\nDataLoader info:")
    print(f"Train batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")

    # Start training
    print("\nStarting training...")
    train(model, train_loader, val_loader, device, num_epochs=30)

In [ ]:
# LinSpace for frames (Not what we want)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import numpy as np
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
import warnings
warnings.filterwarnings("ignore")

# ----------- Dataset Class -----------
class FaceSequenceDataset(Dataset):
    def __init__(self, root_dir, transform=None, max_frames=32, frame_interval=1):
        self.samples = []
        self.transform = transform
        self.label_map = {'original': 0, 'fake': 1}
        self.max_frames = max_frames
        self.frame_interval = frame_interval

        for label in ['original', 'fake']:
            class_dir = os.path.join(root_dir, label)
            if not os.path.exists(class_dir):
                continue
            for folder in os.listdir(class_dir):
                folder_path = os.path.join(class_dir, folder)
                if not os.path.isdir(folder_path):
                    continue
                for person_folder in os.listdir(folder_path):
                    person_path = os.path.join(folder_path, person_folder)
                    if not os.path.isdir(person_path):
                        continue
                    frame_paths = sorted(glob.glob(os.path.join(person_path, '*.jpg')))
                    if len(frame_paths) >= self.max_frames:
                        self.samples.append((frame_paths, self.label_map[label]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        frame_paths, label = self.samples[idx]
        
        # Sample frames uniformly to maintain temporal order
        if len(frame_paths) > self.max_frames:
            # Sample uniformly across the sequence
            indices = np.linspace(0, len(frame_paths)-1, self.max_frames, dtype=int)
            frame_paths = [frame_paths[i] for i in indices]
        
        images = []
        for frame_path in frame_paths:
            try:
                img = Image.open(frame_path).convert('RGB')
                if self.transform:
                    img = self.transform(img)
                images.append(img)
            except Exception as e:
                print(f"Error loading {frame_path}: {e}")
                # Use a black image as fallback
                img = torch.zeros(3, 224, 224)
                images.append(img)
        
        # Pad sequence if needed
        while len(images) < self.max_frames:
            images.append(torch.zeros_like(images[0]))
            
        images = torch.stack(images)  # (T, C, H, W)
        return images, torch.tensor(label, dtype=torch.long)

# ----------- Improved Model with Proper Spatial-Temporal Processing -----------
class SpatialTemporalDeepfakeDetector(nn.Module):
    def __init__(self, num_classes=2, max_frames=32, feature_dim=1408):
        super().__init__()
        
        # Spatial feature extractor
        self.spatial_encoder = EfficientNet.from_pretrained('efficientnet-b2')
        self.spatial_encoder._fc = nn.Identity()
        
        # Spatial feature projection
        self.spatial_projection = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # Temporal modeling with LSTM + Transformer
        self.temporal_lstm = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=2,
            batch_first=True,
            dropout=0.3,
            bidirectional=True
        )
        
        # Temporal attention
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=512,  # bidirectional LSTM output
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        # Positional encoding for temporal sequences
        self.pos_encoding = nn.Parameter(torch.randn(max_frames, 512))
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):  # x: (B, T, C, H, W)
        B, T, C, H, W = x.shape
        
        # Extract spatial features for each frame
        x_reshaped = x.view(B * T, C, H, W)
        spatial_features = self.spatial_encoder.extract_features(x_reshaped)  # (B*T, 1408, h, w)
        
        # Project spatial features
        spatial_features = self.spatial_projection(spatial_features)  # (B*T, 512)
        spatial_features = spatial_features.view(B, T, -1)  # (B, T, 512)
        
        # Add positional encoding
        spatial_features = spatial_features + self.pos_encoding[:T].unsqueeze(0)
        
        # Temporal modeling with LSTM
        lstm_out, _ = self.temporal_lstm(spatial_features)  # (B, T, 512)
        
        # Temporal attention
        attended_features, attention_weights = self.temporal_attention(
            lstm_out, lstm_out, lstm_out
        )  # (B, T, 512)
        
        # Global temporal pooling
        temporal_representation = torch.mean(attended_features, dim=1)  # (B, 512)
        
        # Classification
        output = self.classifier(temporal_representation)
        
        return output

# ----------- Training Utilities -----------
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1

def train(model, train_loader, val_loader, device, num_epochs=30, checkpoint_path='best_model.pth'):
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 1.5]).to(device))  # Weight for class imbalance
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    scaler = GradScaler()

    best_val_f1 = 0.0
    early_stop_counter = 0
    patience = 7

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, all_preds, all_labels = 0, [], []

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, (inputs, labels) in enumerate(progress_bar):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # Update progress bar
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            # Clear cache periodically
            if batch_idx % 50 == 0:
                torch.cuda.empty_cache()

        train_acc, train_prec, train_rec, train_f1 = calculate_metrics(all_labels, all_preds)
        avg_train_loss = train_loss / len(train_loader)
        
        print(f"Train - Loss: {avg_train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}")

        # Validation phase
        model.eval()
        val_loss, all_preds, all_labels = 0, [], []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validating"):
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                
                with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_acc, val_prec, val_rec, val_f1 = calculate_metrics(all_labels, all_preds)
        avg_val_loss = val_loss / len(val_loader)
        
        print(f"Val - Loss: {avg_val_loss:.4f}, Acc: {val_acc:.4f}, Prec: {val_prec:.4f}, Rec: {val_rec:.4f}, F1: {val_f1:.4f}")
        print("-" * 60)

        scheduler.step(avg_val_loss)

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_f1': val_f1,
            'val_loss': avg_val_loss
        }
        torch.save(checkpoint, f"checkpoint_epoch_{epoch+1}.pth")
        
        # Save best model based on F1 score
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(checkpoint, checkpoint_path)
            print(f"New best model saved with F1: {val_f1:.4f}")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping triggered after {patience} epochs without improvement.")
                break
        
        # Clear cache
        torch.cuda.empty_cache()

    print(f"Training completed. Best validation F1: {best_val_f1:.4f}")

# ----------- Setup -----------
if __name__ == '__main__':
    # Paths
    train_dir = '/root/.cache/kagglehub/datasets/binshilin/final-dataset-genreal/versions/1/train'
    test_dir = '/root/.cache/kagglehub/datasets/binshilin/final-dataset-genreal/versions/1/test'

    # Enhanced data augmentation for training
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1)
        ], p=0.7),
        transforms.RandomGrayscale(p=0.05),
        transforms.RandomRotation(10),
        transforms.RandomApply([
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.5))
        ], p=0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
    ])

    # Simple transform for validation
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create datasets
    train_dataset = FaceSequenceDataset(train_dir, transform=train_transform, max_frames=32)
    val_dataset = FaceSequenceDataset(test_dir, transform=val_transform, max_frames=32)

    print(f"Train samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")

    # Create data loaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=2,  # Increased batch size
        shuffle=True, 
        num_workers=4, 
        pin_memory=True,
        drop_last=True
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=2, 
        shuffle=False, 
        num_workers=4, 
        pin_memory=True
    )

    # Initialize model and training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    model = SpatialTemporalDeepfakeDetector().to(device)
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Start training
    train(model, train_loader, val_loader, device, num_epochs=30)